# Li Metal Halide

In [2]:
from ase import Atom, Atoms
from ase.build import mx2, molecule
from ase.data import chemical_symbols
from ase.io import read, write
from ase.spacegroup import get_spacegroup
from ase.visualize import view
from itertools import combinations
import numpy as np
import random

class LiMetalHalidesBuilder:
    """
    用于搭建高对称锂过渡金属卤化物结构的 python 类。
    """
    def __init__(self, prototype="LMX3", m_element="Zn", x_element="Cl", 
                 lattice_a=0.0, lattice_b=0.0, lattice_c=0.0, supercell_matrix=(2, 2, 2), **kwargs):
        self.prototype = prototype
        self.m_element = m_element
        self.x_element = x_element
        self.lattice_a = lattice_a
        self.lattice_b = lattice_b
        self.lattice_c = lattice_c
        self.supercell_matrix = supercell_matrix
        
    def predefined_systems(self):
        predefined_dict = {
            "LMX3": {
                "symbols": ["Li"] * 1 + [self.m_element] * 1 + [self.x_element] * 3,
                "cell": [6.09280, 3.69900, 6.33590, 90.0000, 115.3438, 90.0000],
                "scaled_positions": [
                    [0.5046200159999999, 0.5000000000000000, 0.6633399720000002], # Li1
                    [0.1893499940000000, 0.0000000000000000, 0.0669300029999999], # Zn1
                    [0.4088900090000001, 0.5000000000000000, 0.0281700010000000], # Cl1
                    [0.8102700110000000, 0.0000000000000000, 0.7798799870000001], # Cl2
                    [0.2189600020000000, 0.0000000000000000, 0.4343500139999999], # Cl3
                ]
            },
            "L2MX4": {
                "symbols": ["Li"] * 2 + [self.m_element] * 1 + [self.x_element] * 4,
                "cell": [6.39538, 6.39538, 3.65470, 90.0000, 90.0000, 109.6669],
                "scaled_positions": [
                    [0.0000000000000000, 0.5000000000000000, 0.5000000000000000], # Li1
                    [0.5000000000000000, 0.0000000000000000, 0.5000000000000000], # Li2
                    [0.0000000000000000, 0.0000000000000001, 0.0000000000000000], # Mg1
                    [0.7658799890000000, 0.7658799890000001, 0.5000000000000000], # Cl1
                    [0.2341200110000000, 0.2341200110000001, 0.5000000000000000], # Cl2
                    [0.2391400190000001, 0.7608599810000000, 0.0000000000000000], # Cl3
                    [0.7608599809999999, 0.2391400190000001, 0.0000000000000000], # Cl4
                ]
            },
            "L3MX6": {
                "symbols": ["Li"] * 3 + [self.m_element] * 1 + [self.x_element] * 6,
                "cell": [6.26410, 6.26410, 6.24320, 90.0000, 90.0000, 120.0000],
                "scaled_positions": [
                    [0.3333333333333333, 0.6666666666666666, 0.5000000000000000], # Li1
                    [0.6666666666666667, 0.3333333333333334, 0.5000000000000000], # Li2
                    [0.0000000000000000, 0.0000000000000000, 0.0000000000000000], # Li3
                    [0.0000000000000000, 0.0000000000000000, 0.5000000000000000], # Al1
                    [0.3037300110000000, 0.3037300110000000, 0.7190799710000002], # Cl1
                    [0.6962699889999999, 0.6962699889999999, 0.2809200289999998], # Cl2
                    [0.6962699889999999, 0.0000000000000000, 0.7190799710000002], # Cl3
                    [0.3037300110000000, 0.0000000000000000, 0.2809200289999998], # Cl4
                    [0.0000000000000000, 0.6962699889999999, 0.7190799710000002], # Cl5
                    [0.0000000000000000, 0.3037300110000000, 0.2809200289999998], # Cl6
                ]
            },
            "L6MX8": {
                "symbols": ["Li"] * 6 + [self.m_element] * 1 + [self.x_element] * 8,
                "cell": [7.31983, 7.31983, 7.31983, 60.0000, 60.0000, 60.0000],
                "scaled_positions": [
                    [0.5000000000000000, 0.0000000000000000, 0.5000000000000000], # Li1
                    [0.0000000000000000, 0.5000000000000000, 0.0000000000000000], # Li2
                    [0.5000000000000000, 0.5000000000000000, 0.0000000000000000], # Li3
                    [0.0000000000000000, 0.0000000000000000, 0.5000000000000000], # Li4
                    [0.0000000000000000, 0.5000000000000000, 0.5000000000000000], # Li5
                    [0.5000000000000000, 0.0000000000000000, 0.0000000000000000], # Li6
                    [0.0000000000000000, 0.0000000000000000, 0.0000000000000000], # Mg1
                    [0.7585300210000000, 0.2414699790000000, 0.7585300210000000], # Cl1
                    [0.2414699790000001, 0.7585300209999999, 0.2414699790000001], # Cl2
                    [0.7585300209999999, 0.7585300209999999, 0.2414699790000001], # Cl3
                    [0.2414699790000001, 0.2414699790000001, 0.7585300209999999], # Cl4
                    [0.2414699790000001, 0.7585300210000000, 0.7585300210000000], # Cl5
                    [0.7585300209999999, 0.2414699790000000, 0.2414699790000000], # Cl6
                    [0.2500000000000000, 0.2500000000000000, 0.2500000000000000], # Cl7
                    [0.7500000000000000, 0.7500000000000000, 0.7500000000000000], # Cl8
                ]
            },
        }
        return predefined_dict

    def build(self):
        lmh_dict = self.predefined_systems()[self.prototype]
        cell = [
            self.lattice_a if self.lattice_a != 0 else lmh_dict["cell"][0],
            self.lattice_b if self.lattice_b != 0 else lmh_dict["cell"][1],
            self.lattice_c if self.lattice_c != 0 else lmh_dict["cell"][2],
            lmh_dict["cell"][3],
            lmh_dict["cell"][4],
            lmh_dict["cell"][5],
        ]
        structure = Atoms(symbols=lmh_dict["symbols"], scaled_positions=lmh_dict["scaled_positions"], cell=cell, pbc=True)
        return structure * self.supercell_matrix

In [3]:
def identify_system_symmetry(atoms):
    symmetry = get_spacegroup(atoms, symprec=1e-5)
    international_symbol = symmetry.symbol
    spacegroup_number = symmetry.no
    return international_symbol, spacegroup_number
    
system = LiMetalHalidesBuilder(prototype="LMX3")
atoms = system.build()
print(identify_system_symmetry(atoms))
view(atoms)
# write("mbene.vasp", atoms, direct=True, sort=True, vasp5=True)

('P m', 6)


<Popen: returncode: None args: ['C:\\Users\\wangchangrui\\.conda\\envs\\Q-Ap...>

## Q-Api 接口

In [ ]:
def li_metal_halides_builder(**kwargs):
    kwargs['supercell_matrix'] = (kwargs['n_a'], kwargs['n_b'], kwargs['n_c'])
    system = LiMetalHalidesBuilder(**kwargs)
    new_structure = system.build()
    cif_output = io.BytesIO()
    write(cif_output, new_structure, format='cif')
    cif_string = cif_output.getvalue().decode('utf-8')
    return cif_string

In [ ]:
@guide_register_func(ModelTag('构建高对称锂过渡金属卤化物结构', ['电池'], crystal_type='ionic crystal', note='锂金属卤化物 (LMX) 作为一类潜在的固态电解质 (SSE) 越来越有吸引力，以实现高性能全固态电池 (ASSB)，因为它们具有高氧化电位、良好的延展性、和合理的湿度恢复能力。').identifier)
def TrandionMetalHalides(data):
    mode = data.get('mode')
    if mode == 'init':
        title = Description(name='title', note='构建高对称二维过渡金属卤化物结构，需要指定体相基体元素及晶格参数等信息')
        prototype = SingleFromList(name='prototype', 
                                   note='体相基体类型', 
                                   id='prototype', 
                                   list_value=[{'label': i, 'value': i} for i in ['LMX3', 'L2MX4', 'L3MX6', 'L6MX8']], 
                                   default_value='LMX3', 
                                   is_required=0)
        m_element = SingleFromList(name='m_element', 
                                   note='体相基体中的 M 元素', 
                                   id='m_element', 
                                   list_value=[{'label': '镁', 'value': 'Mg'},
                                               {'label': '铝', 'value': 'Al'},
                                               {'label': '钙', 'value': 'Ca'},
                                               {'label': '钡', 'value': 'Ba'},
                                               {'label': '钛', 'value': 'Ti'},
                                               {'label': '铜', 'value': 'Cu'},
                                               {'label': '锌', 'value': 'Zn'},
                                               {'label': '钴', 'value': 'Co'},
                                               {'label': '锰', 'value': 'Mn'},
                                               {'label': '镍', 'value': 'Ni'},
                                               {'label': '铁', 'value': 'Fe'}],
                                   default_value='Zn',
                                   is_required=0)
        x_element = SingleFromList(name='x_element', 
                                   note='体相基体中的 X 元素', 
                                   id='x_element', 
                                   list_value=[{'label': '氯', 'value': 'Cl'},
                                               {'label': '溴', 'value': 'Br'}],
                                   default_value='Cl',
                                   is_required=0)
        lattice_a = SingleInput(name='lattice_a', note='a 方向晶格常数 (Å)', id='lattice_a', input_type='float', default_value=0.0, min=0.0, max=100.0, is_required=0)
        lattice_b = SingleInput(name='lattice_b', note='b 方向晶格常数 (Å)', id='lattice_b', input_type='float', default_value=0.0, min=0.0, max=100.0, is_required=0)
        lattice_c = SingleInput(name='lattice_c', note='c 方向晶格常数 (Å)', id='lattice_c', input_type='float', default_value=0.0, min=0.0, max=100.0, is_required=0)
        n_a = SingleInput(name='n_a', note='a 轴的单胞重复单元', id='n_a', input_type='int', default_value=3, min=1, max=10, is_required=0)
        n_b = SingleInput(name='n_b', note='b 轴的单胞重复单元', id='n_b', input_type='int', default_value=3, min=1, max=10, is_required=0)
        n_c = SingleInput(name='n_c', note='c 轴的单胞重复单元', id='n_c', input_type='int', default_value=3, min=1, max=10, is_required=0)

        return [title(), prototype(), m_element(), x_element(), lattice_a(), lattice_b(), lattice_c(), n_a(), n_b(), n_c()]
        
    elif mode == 'generate':
        value = data.get('value')
        try:
            cif_string = trandion_metal_halides_builder(**value)
        except Exception as e:
            return {'error':str(e)}
        
        return {'file_content': cif_string, 'file_format': 'cif'}